In [49]:
import yfinance as yf
import pandas as pd
import os

In [50]:
if os.path.exists("sp500.csv"):
    sp500 = pd.read_csv("sp500.csv", index_col=0)
else:
    sp500 = yf.Ticker("^GSPC")
    sp500 = sp500.history(period="max")
    sp500.to_csv("sp500.csv")

In [51]:
sp500.index = pd.to_datetime(sp500.index)

C:\Users\naimk\AppData\Local\Temp\ipykernel_24712\318495744.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  sp500.index = pd.to_datetime(sp500.index)


In [52]:
sp500

,Open,High,Low,Close,Volume,NextClose
Date,,,,,,
2024-07-16,"5,644.09","5,669.67","5,639.02","5,667.20",NaN,NaN
2024-07-15,"5,638.16","5,666.94","5,614.75","5,631.22",NaN,NaN
2024-07-12,"5,590.76","5,655.56","5,590.44","5,615.35",NaN,NaN
2024-07-11,"5,635.21","5,642.45","5,576.53","5,584.54",NaN,NaN
2024-07-10,"5,591.26","5,635.39","5,586.44","5,633.91",NaN,NaN
...,...,...,...,...,...,...
1950-09-01,17.07999992,17.07999992,17.07999992,17.07999992,2520000.0,NaN
1950-06-01,16.97999954,16.97999954,16.97999954,16.97999954,2010000.0,NaN
1950-05-01,16.93000031,16.93000031,16.93000031,16.93000031,2550000.0,NaN


In [53]:
sp500["Close"] = sp500["Close"].str.replace(',', '').astype(float)


In [54]:
sp500["Tomorrow"] = sp500["Close"].shift(-1)

In [55]:
sp500["Target"] = (sp500["Tomorrow"] > sp500["Close"]).astype(int)

In [56]:
sp500 = sp500.sort_index()
sp500 = sp500.loc["1990-01-01":].copy()

In [57]:
for col in ["Close", "Volume", "Open", "High", "Low"]:
    sp500[col] = sp500[col].astype(str).str.replace(',', '').astype(float)


In [58]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score
# Define the necessary variables
model = RandomForestClassifier(n_estimators=200, min_samples_split=50, random_state=1)
test = sp500  # Replace with the appropriate test data
new_predictors = []  # Define the "new_predictors" variable here
predictors = new_predictors  # Replace with the appropriate predictors

# Check if the 'estimators_' attribute exists before making predictions
if hasattr(model, 'estimators_'):
    preds = model.predict(test[predictors])
    preds = pd.Series(preds, index=test.index)
    precision_score(test["Target"], preds)
else:
    print("The 'estimators_' attribute does not exist for the RandomForestClassifier object.")


The 'estimators_' attribute does not exist for the RandomForestClassifier object.


In [59]:
if 'Target' in locals() and 'train' not in locals():
    train = pd.DataFrame()  # Replace with the appropriate code to define the "train" variable

if 'Target' in locals() and 'train' in locals() and 'Target' in train.columns:
    model.fit(train[predictors], train["Target"])



In [60]:
def predict(train, test, predictors, model):
    model.fit(train[predictors], train["Target"])
    preds = model.predict(test[predictors])
    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["Target"], preds], axis=1)
    return combined

In [61]:
def backtest(data, model, predictors, start=2500, step=250):
    all_predictions = []

    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+step)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)
    
    return pd.concat(all_predictions)

In [62]:
# Ensure that the necessary variables are defined and have the correct data types
sp500 = sp500.dropna(subset=["Target"])
model = RandomForestClassifier(n_estimators=200, min_samples_split=50, random_state=1)
predictors = ["Close", "Volume", "Open", "High", "Low"]  # Replace with the appropriate predictors

# Call the backtest function with the updated variables
predictions = backtest(sp500, model, predictors)

In [63]:
predictions["Predictions"].value_counts()

Predictions
0    3169
1    3031
Name: count, dtype: int64

In [64]:
from sklearn.metrics import precision_score
precision_score(predictions["Target"], predictions["Predictions"])

np.float64(0.6083800725833058)

In [65]:
predictions["Target"].value_counts() / predictions.shape[0]

Target
0    0.535806
1    0.464194
Name: count, dtype: float64

In [66]:
horizons = [2,5,60,250,1000]
new_predictors = []

for horizon in horizons:
    rolling_averages = sp500.rolling(horizon).mean()
    
    ratio_column = f"Close_Ratio_{horizon}"
    sp500[ratio_column] = sp500["Close"] / rolling_averages["Close"]
    
    trend_column = f"Trend_{horizon}"
    sp500[trend_column] = sp500.shift(1).rolling(horizon).sum()["Target"]
    
    new_predictors+= [ratio_column, trend_column]

In [67]:
sp500 = sp500.dropna(subset=sp500.columns[sp500.columns != "Tomorrow"])

In [68]:
sp500

,Open,High,Low,Close,Volume,NextClose,Tomorrow,Target,Close_Ratio_2,Trend_2,Close_Ratio_5,Trend_5,Close_Ratio_60,Trend_60,Close_Ratio_250,Trend_250,Close_Ratio_1000,Trend_1000
Date,,,,,,,,,,,,,,,,,,


In [69]:
model = RandomForestClassifier(n_estimators=200, min_samples_split=50, random_state=1)

In [70]:
def predict(train, test, predictors, model):
    model.fit(train[predictors], train["Target"])
    preds = model.predict_proba(test[predictors])[:,1]
    preds[preds >=.6] = 1
    preds[preds <.6] = 0
    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["Target"], preds], axis=1)
    return combined

In [71]:
concatenated_predictions = pd.concat([predictions])


In [72]:
predictions["Predictions"].value_counts()

Predictions
0    3169
1    3031
Name: count, dtype: int64

In [73]:
precision_score(predictions["Target"], predictions["Predictions"])

np.float64(0.6083800725833058)

In [74]:
predictions["Target"].value_counts() / predictions.shape[0]

Target
0    0.535806
1    0.464194
Name: count, dtype: float64

In [75]:
predictions

,Target,Predictions
Date,,
1999-11-23,1,1
1999-11-24,0,0
1999-11-26,1,1
1999-11-29,1,1
1999-11-30,1,0
...,...,...
2024-07-10,0,1
2024-07-11,1,1
2024-07-12,0,1
